In [1]:
import re
import os
import pandas as pd
from PyPDF2 import PdfReader
import easyocr
import numpy as np
from pdf2image import convert_from_path
import pymorphy2
from pymorphy2 import MorphAnalyzer

In [2]:
morph = MorphAnalyzer()

In [3]:
reader_ml = easyocr.Reader(['ru', 'en'], gpu=True)  


def get_info_from_pdf(pdf: 'str'): 
    """На вход функции подаётся адрес до pdf-документа"""
    
    

    # ШАГ 1: пробуем распознать "digitally-created PDF":
    pdf_text_list = []
    # ридер из PyPDF2:
    reader = PdfReader(pdf)

    # собираем информацию с каждой страницы pdf-документа...:
    number_of_pages = len(reader.pages)
    for i in range(number_of_pages):
        page = reader.pages[i]
        try:
            text = page.extract_text()        
            if text != '': # если текст на странице НЕ пустой
                text = re.split('/n|\n', text)  # делим построчно, сохраняя структуру документа
                pdf_text_list.extend(text) # добавляем в единый список текста всей pdf
        except:
            pdf_text_list = []

    # если полученный список не пустой...:
    if pdf_text_list != []:
        is_digit = True
        good_pdf_quality = True
        pdf_text_info = pdf_text_list

    # ШАГ 2: если это не цифровая pdf, а картинка - исползуем EasyOCR:
    else:
        is_digit = False

        regular_pattern = re.compile(r'\w\w')

        # ридер из EasyOCR:
        image = convert_from_path(pdf)
        image_text_list = []
        for page in range(len(image)):
            page_np_array = np.array(image[page])
            text = reader_ml.readtext(page_np_array, detail = 0, paragraph=True) 
            image_text_list.extend(text)
        
        pdf_text_info = image_text_list
        

        # оцениваем качество распознвания EasyOCR:
        text_recognition_score = []
        for string in image_text_list:
            count = 0
            bad = 0
            for word in regular_pattern.findall(string):
                count += 1
                parsed_info = morph.parse(word)[0]
                bad += (not parsed_info.tag.POS or 
                    isinstance(parsed_info.methods_stack[0][0], pymorphy2.units.by_analogy.KnownSuffixAnalyzer.FakeDictionary)) # FakeDictionary - значит, что текст не распознан
            try:
                text_recognition_score.append(bad/count) # по каждой строке добавялем скор-отношение плохо распознанных слов к количеству всех слов
            except:
                text_recognition_score.append(0.5) # если добавить не получилось - добавляем среднее значение 0.5


        # высчитываем средний скор по всем строкам:
        avg_text_recognition_score = sum(text_recognition_score) / len(text_recognition_score)
        if avg_text_recognition_score > 0.75:
            good_pdf_quality = False
        else:
            good_pdf_quality = True


    return pdf_text_info, is_digit, good_pdf_quality

In [4]:
from tqdm import tqdm

In [6]:
base_path = '/home/admin01/vadim/glossary/data/Канцелярия'
result_path = '/home/admin01/vadim/glossary/data/Результат'
datafolder_collection = os.listdir(base_path)

for datafolder in datafolder_collection:

    path_to_datafolder = os.path.join(base_path, datafolder) # папка с пдф-ками по задаче глоссария

    result = {'pdf': [],
            'Текст из pdf': [],
            'Цифровая pdf?': [],
            'Хорошее качество текста?': []}


    for pdf_name in tqdm(os.listdir(path_to_datafolder)):
        pdf_name = path_to_datafolder + '/' + pdf_name

        try:
            pdf_text_info, is_digit, good_pdf_quality = get_info_from_pdf(pdf_name)
        
        except:
            pdf_text_info = ['Ошибка считывания']
            good_pdf_quality = None
            is_digit = None
            print(pdf_name, '- тут живет ошибка')

        
        result['pdf'].append(pdf_name)
        result['Текст из pdf'].append(' '.join(pdf_text_info)) # обхединяем в строку: 1 строка - 1 целый pdf-документ
        result['Цифровая pdf?'].append(is_digit)
        result['Хорошее качество текста?'].append(good_pdf_quality)

    df = pd.DataFrame(result)
    
    df.to_csv(os.path.join(result_path, datafolder + '.csv' ))

 16%|█▌        | 53/333 [03:35<05:29,  1.18s/it]  

/home/admin01/vadim/glossary/Канцелярия/Приказы по АХД  Дело 02 2014 год/Thumbs.db - тут живет ошибка


 82%|████████▏ | 273/333 [20:21<06:20,  6.35s/it]  

/home/admin01/vadim/glossary/Канцелярия/Приказы по АХД  Дело 02 2014 год/Приложения к 02-191-14 - тут живет ошибка


 16%|█▌        | 231/1435 [00:12<00:49, 24.23it/s]

...

 60%|█████▉    | 2454/4124 [1:32:49<44:25,  1.60s/it]   

/home/admin01/vadim/glossary/Канцелярия/Все приказы 2011 года/Ярлык для 01-2502-11.lnk - тут живет ошибка


 84%|████████▎ | 3452/4124 [2:52:25<40:11,  3.59s/it]   

/home/admin01/vadim/glossary/Канцелярия/Все приказы 2011 года/01-2244-11 (2).pdf - тут живет ошибка


100%|██████████| 4124/4124 [4:02:20<00:00,  3.53s/it]  
